# Importing Dependencies

In [96]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import pandas_datareader.data as web
import schwabdev
from scipy import stats
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pprint
%matplotlib inline

# Momentum Investing
- the strategy in investing in the stocks that increased in price the most

# Collecting S&P 500 constituents

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_constituents = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [4]:
sp500_constituents = sp500_constituents['Symbol'].to_list()

### changing format to match Schwab's

In [5]:
sp500_constituents = [x.replace('.', '/') for x in sp500_constituents]

# Setting up Schwab API

In [6]:
from key import api_key, api_secret

In [7]:
client = schwabdev.Client(app_key=api_key, app_secret=api_secret, capture_callback=False)

# Creating Datafram
- Basing this strategy in a 6 month change

In [8]:
my_columns = ['Ticker', 'Price (6 months)', 'Price (now)', 'Net Change', 'Number of Shares to Buy']

In [9]:
today = datetime.today()
six_months = today - relativedelta(months = 6)

In [10]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', period = 6, startDate=six_months, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_6_months = price_history['candles'][0]['close']
    net_change = round(((price_now - price_6_months)/ price_6_months) * 100, 2)
    entries.append(
        [
            symbol,
            price_6_months,
            price_now,
            net_change,
            'N/a'
        ]
    )

In [11]:
momentum_df = pd.DataFrame(entries, columns=my_columns)

### Sorting by biggest increases

In [12]:
momentum_df.sort_values(by='Net Change', ascending=False, inplace=True)

In [13]:
momentum_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
360,PLTR,44.86,93.40,108.20,N/a
308,MCK,507.41,700.70,38.09,N/a
171,EQT,37.48,51.61,37.70,N/a
468,VRSN,181.26,247.07,36.31,N/a
433,TTWO,161.79,214.44,32.54,N/a


# Creating Excel Output with all data

In [14]:
momentum_df_copy = momentum_df.copy()

In [15]:
momentum_df_copy = momentum_df_copy.drop('Number of Shares to Buy', axis=1)
momentum_df_copy.head()

,Ticker,Price (6 months),Price (now),Net Change
360,PLTR,44.86,93.40,108.20
308,MCK,507.41,700.70,38.09
171,EQT,37.48,51.61,37.70
468,VRSN,181.26,247.07,36.31
433,TTWO,161.79,214.44,32.54


In [55]:
writer = pd.ExcelWriter('total_data.xlsx', engine='xlsxwriter')

momentum_df_copy.to_excel(writer, sheet_name='Movers', index=False)

### Formatting

In [56]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [57]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [58]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying Formats

In [59]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format]
}

In [60]:
for column in column_formats.keys():
    writer.sheets['Movers'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Movers'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

In [61]:
writer.close()

# Keeping Top 50 Movers

In [23]:
final_df = momentum_df.copy()

In [24]:
final_df = final_df.head(50)
final_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 360 to 334
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   50 non-null     object 
 1   Price (6 months)         50 non-null     float64
 2   Price (now)              50 non-null     float64
 3   Net Change               50 non-null     float64
 4   Number of Shares to Buy  50 non-null     object 
dtypes: float64(3), object(2)
memory usage: 2.3+ KB


In [25]:
final_df.reset_index(inplace=True)
final_df.drop('index', axis = 1, inplace=True)
final_df

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,44.86,93.4000,108.20,N/a
1,MCK,507.41,700.7000,38.09,N/a
2,EQT,37.48,51.6100,37.70,N/a
3,VRSN,181.26,247.0700,36.31,N/a
4,TTWO,161.79,214.4400,32.54,N/a
5,PAYC,164.63,216.4200,31.46,N/a
6,NFLX,754.68,979.7050,29.82,N/a
7,TKO,115.25,149.2650,29.51,N/a
8,CNP,29.31,37.5550,28.13,N/a
9,DRI,159.92,202.7700,26.79,N/a


# Calculating the Number of Shares to Buy

In [26]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of the porfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print('Invalid input! \n Please try again.')
        portfolio_size = input('Enter the size of the porfolio: ')


In [27]:
portfolio_input()

In [28]:
print(portfolio_size)

100000


In [29]:
position_size = float(portfolio_size) / len(final_df.index)
position_size

2000.0

In [30]:
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Price (now)'])

In [31]:
final_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,44.86,93.40,108.20,21
1,MCK,507.41,700.70,38.09,2
2,EQT,37.48,51.61,37.70,38
3,VRSN,181.26,247.07,36.31,8
4,TTWO,161.79,214.44,32.54,9


# Exporting to Excel

In [79]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

final_df.to_excel(writer, sheet_name='Recommended Trades', index=False)

In [80]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [81]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [82]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [83]:
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [84]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format],
    'E': ['Number of Shares to Buy', integer_format]
}

In [85]:
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [86]:
writer.close()

# More Realistic Momentum Strategy

Want to differentiate from high quality and low quality
- high quality - have slow and steady outperformance over long period of time
- low quality - might be from one surge of increase 

We want high quality momentum stocks because it shows repeated success rather than a surge in increase due to one event.

In [87]:
hqm_columns = [
    'Ticker',
    'Price',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]

hqm_df = pd.DataFrame(columns=hqm_columns)
hqm_df

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile


## Making API Calls

In [101]:
today = datetime.today()
one_year = today - relativedelta(years = 1, months = 1)

In [102]:
print(today)
print(one_year)

2025-04-17 13:36:07.577227
2024-03-17 13:36:07.577227


In [103]:
symbol = 'AAPL'
price_history = client.price_history(symbol, periodType='year', frequencyType='monthly', frequency=1, startDate=one_year, endDate=today).json()

In [106]:
print(price_history['candles'][6]['datetime'])

1727758800000


In [107]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', frequency=1, startDate=one_year, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_12_months = price_history['candles'][0]['close']
    price_6_months = price_history['candles'][6]['close']
    price_3_months = price_history['candles'][9]['close']
    price_1_month = price_history['candles'][-2]['close']
    net_change_12= round(((price_now - price_12_months)/ price_12_months) * 100, 2)
    net_change_6 = round(((price_now - price_6_months)/ price_6_months) * 100, 2)
    net_change_3 = round(((price_now - price_3_months)/ price_3_months) * 100, 2)
    net_change_1 = round(((price_now - price_1_month)/ price_1_month) * 100, 2)
    entries.append(
        [
            symbol,
            price_now,
            net_change_12,
            'N/a',
            net_change_6,
            'N/a',
            net_change_3,
            'N/a',
            net_change_1,
            'N/a'
        ]
    )

In [131]:
hqm_df = pd.DataFrame(entries, columns=hqm_columns)

In [132]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,132.0300,47.88,N/a,35.90,N/a,32.47,N/a,-2.88,N/a
1,AOS,63.4900,-28.89,N/a,-24.69,N/a,-25.11,N/a,-1.57,N/a
2,ABT,132.3700,19.72,N/a,25.00,N/a,27.34,N/a,4.33,N/a
3,ABBV,174.7425,-2.08,N/a,6.69,N/a,11.26,N/a,-0.18,N/a
4,ACN,284.7200,-15.64,N/a,-6.25,N/a,-5.41,N/a,0.13,N/a


## Calculating Momentum Percentiles 

In [133]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for period in time_periods:
        hqm_df.loc[row, f'{period} Return Percentile'] = stats.percentileofscore(
            hqm_df[f'{period} Price Return'],
            hqm_df.loc[row, f'{period} Price Return']
        )

In [134]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,MMM,132.0300,47.88,95.029821,35.90,91.848907,32.47,91.252485,-2.88,7.952286
1,AOS,63.4900,-28.89,15.308151,-24.69,15.705765,-25.11,14.71173,-1.57,15.705765
2,ABT,132.3700,19.72,79.324056,25.00,83.101392,27.34,86.878728,4.33,90.059642
3,ABBV,174.7425,-2.08,48.310139,6.69,59.642147,11.26,70.178926,-0.18,32.902584
4,ACN,284.7200,-15.64,30.815109,-6.25,38.568588,-5.41,39.960239,0.13,38.071571


## Calculating HQM Score
- HQM Score - mean of the four percentiles scores

In [135]:
from statistics import mean

In [136]:
for i in range(len(hqm_df)):
    momentum_percentiles = []
    for period in time_periods:
        momentum_percentiles.append(hqm_df.loc[i, f'{period} Return Percentile'])
    hqm_df.loc[i, 'HQM Score'] = mean(momentum_percentiles)

In [137]:
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,132.0300,47.88,95.029821,35.90,91.848907,32.47,91.252485,-2.88,7.952286,71.520875
1,AOS,63.4900,-28.89,15.308151,-24.69,15.705765,-25.11,14.71173,-1.57,15.705765,15.357853
2,ABT,132.3700,19.72,79.324056,25.00,83.101392,27.34,86.878728,4.33,90.059642,84.840954
3,ABBV,174.7425,-2.08,48.310139,6.69,59.642147,11.26,70.178926,-0.18,32.902584,52.758449
4,ACN,284.7200,-15.64,30.815109,-6.25,38.568588,-5.41,39.960239,0.13,38.071571,36.853877


### Getting top performing stocks

In [138]:
hqm_df.sort_values(by='HQM Score', ascending=False, inplace=True)
hqm_df.head()

,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
360,PLTR,93.5513,286.90,100.0,300.99,100.0,345.27,100.0,5.65,95.626243,98.906561
442,TPL,1307.0100,136.80,99.602386,136.94,99.801193,117.38,99.801193,5.74,96.023857,98.807157
371,PM,163.9950,80.45,99.204771,68.37,99.005964,64.11,99.204771,6.57,97.614314,98.757455
435,TRGP,177.7650,60.29,97.017893,58.28,97.415507,55.55,98.011928,8.68,99.602386,98.011928
333,NFLX,976.5300,55.50,96.620278,68.56,99.204771,50.99,97.415507,6.34,97.017893,97.564612


### exporting to excel

In [139]:
writer = pd.ExcelWriter('total_data.xlsx', engine='xlsxwriter')

hqm_df.to_excel(writer, sheet_name='Better Movers', index=False)

In [140]:
hqm_column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['One-Year Price Return', percent_format],
    'D': ['One-Year Return Percentile', percent_format],
    'E': ['Six-Month Price Return', percent_format],
    'F': ['Six-Month Return Percentile', percent_format],
    'G': ['Three-Month Price Return', percent_format],
    'H': ['Three-Month Return Percentile', percent_format],
    'I': ['One-Month Price Return', percent_format],
    'J': ['One-Month Return Percentile', percent_format],
    'K': ['HQM Score', percent_format]
}

In [141]:
for column in column_formats.keys():
    writer.sheets['Better Movers'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Better Movers'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [142]:
writer.close()